In [13]:
from pyspark.sql import SparkSession

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://ben-spark-master:7077") \
        .appName("shared_variables")\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

# Old API (RDD)

spark_context = spark_session.sparkContext

# Immutable data for all workers
broadcast_variable = spark_context.broadcast({'apples': 100, 'oranges': 42, 'pears': 1})

In [14]:

rdd = spark_context.parallelize(['apples', 'pears'])

# Convert fruit to amounts of fruit.
# 'value' is captured in the closure, but its only a handle - the dictionary won't be serialized
result = rdd.map(lambda x: broadcast_variable.value[x])\
            .collect()

print(result)

#See: http://spark.apache.org/docs/2.3.0/api/python/pyspark.html

[100, 1]


In [16]:
accumulator_1 = spark_context.accumulator(0)

rdd2 = spark_context.parallelize([1,2,3,4,5,6],2)

result = rdd2.foreach(lambda x: accumulator_1.add(1))

print(accumulator_1.value)

6


In [12]:
# release the cores for another application!
spark_context.stop()